In [4]:
#! pip install pyspark
#! pip install pyarrow

In [47]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, date_format, lit

import pandas as pd

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" 
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

spark

In [6]:
attend = spark.read.csv('attend.csv', header=True, inferSchema=True)
attend.printSchema()

root
 |-- уникальный номер занятия: integer (nullable = true)
 |-- уникальный номер группы: integer (nullable = true)
 |-- уникальный номер участника: integer (nullable = true)
 |-- направление 2: string (nullable = true)
 |-- направление 3: string (nullable = true)
 |-- онлайн/офлайн: string (nullable = true)
 |-- дата занятия: date (nullable = true)
 |-- время начала занятия: timestamp (nullable = true)
 |-- время окончания занятия: timestamp (nullable = true)



In [23]:
attend = attend.withColumn("дата занятия",to_timestamp(col("дата занятия"))).withColumn("month", date_format(col("дата занятия"), "M"))
attend = attend.withColumn("1", lit(1))
attend.show()

+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----+---+
|уникальный номер занятия|уникальный номер группы|уникальный номер участника|       направление 2|       направление 3|онлайн/офлайн|       дата занятия|время начала занятия|время окончания занятия|month|  1|
+------------------------+-----------------------+--------------------------+--------------------+--------------------+-------------+-------------------+--------------------+-----------------------+-----+---+
|               401346550|              801346550|                 101352023|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|2022-08-01 00:00:00| 2023-05-27 09:00:00|    2023-05-27 10:00:00|    8|  1|
|               401346550|              801346550|                 101385462|   ОНЛАЙН Гимнастика|        ОНЛАЙН Цигун|           Да|2022-08-01 00:00:00| 2023-05-27

In [24]:
pivot_1 = attend.groupBy("направление 2").pivot("month").sum('1')
pivot_1.show()

+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|       направление 2|    1|   10|   11|   12|    2|    4|    5|    6|    7|    8|    9|
+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|                 ОФП|16060|17074|19792|17627|18703|11627| 9289| 8717| 6223| 7327|11807|
|   Иностранные языки| 5156| 4363| 5437| 4764| 5867| 1882| 1371|  972|  452|  544| 2052|
|          Киберспорт| null|   10|    8|    1|    1|    4|    1|    4| null|   19|   15|
|               Пение| 9153|10497|11795|10399|11421| 5894| 4816| 4523| 2975| 4050| 6957|
|ОНЛАЙН Художестве...|14889|14165|16573|16874|17783|14646|11722|11472| 9749|10360|12128|
|ОНЛАЙН Образовате...|15630|16208|17770|16881|18010|15826|12262|12153|10422|11517|14333|
|ОНЛАЙН Английский...|13415|14726|16315|16171|14907|19114|14461|11535| 8580| 8501|12556|
|    ОНЛАЙН Рисование|21511|22333|24782|25684|25197|21472|16997|16612|14897|17903|20666|
|ОНЛАЙН Шахматы и ...

In [25]:
pivot_2 = attend.groupBy("направление 3").pivot("month").sum('1')
pivot_2.show()

+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|       направление 3|    1|   10|   11|   12|    2|    4|   5|   6|   7|   8|    9|
+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|Мастер-класс по у...|  603|  984|  956|  818|  818|  192| 223| 413| 313| 430|  582|
|ОНЛАЙН Греческий ...| null| null| null| null| null|   60|  80|null|null|null| null|
|Архитектура Москв...|   94|   72|  122|   90|  100|   68|  50|  51|null|null|   14|
|ОНЛАЙН География....| 4251| 4702| 4862| 4007| 4724| 4013|2898|2719|3242|3651| 4244|
|Искусство коммуни...|  176|  153|  168|  139|  182|   61|  70|  65|  19|  45|   75|
|   История искусства|  766|  825| 1090|  952| 1210|  138| 269| 334| 362| 426|  703|
|Вышивка гладью и ...|    2|    9|    8|    4|    9| null|null|   1|null|null|    1|
|  Интернет-маркетинг|   64| null| null|   49|   59| null|null|null|null|null| null|
|ОНЛАЙН История, к...| 2934| 3195| 3690| 3679| 3682| 2889|2423|23

In [26]:
pivot_1 = pivot_1.withColumnRenamed("направление 2","направление")
pivot_1.show()

+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|         направление|    1|   10|   11|   12|    2|    4|    5|    6|    7|    8|    9|
+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|                 ОФП|16060|17074|19792|17627|18703|11627| 9289| 8717| 6223| 7327|11807|
|   Иностранные языки| 5156| 4363| 5437| 4764| 5867| 1882| 1371|  972|  452|  544| 2052|
|          Киберспорт| null|   10|    8|    1|    1|    4|    1|    4| null|   19|   15|
|               Пение| 9153|10497|11795|10399|11421| 5894| 4816| 4523| 2975| 4050| 6957|
|ОНЛАЙН Художестве...|14889|14165|16573|16874|17783|14646|11722|11472| 9749|10360|12128|
|ОНЛАЙН Образовате...|15630|16208|17770|16881|18010|15826|12262|12153|10422|11517|14333|
|ОНЛАЙН Английский...|13415|14726|16315|16171|14907|19114|14461|11535| 8580| 8501|12556|
|    ОНЛАЙН Рисование|21511|22333|24782|25684|25197|21472|16997|16612|14897|17903|20666|
|ОНЛАЙН Шахматы и ...

In [28]:
pivot_2 = pivot_2.withColumnRenamed("направление 3","направление")
pivot_2.show()

+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|         направление|    1|   10|   11|   12|    2|    4|   5|   6|   7|   8|    9|
+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|Мастер-класс по у...|  603|  984|  956|  818|  818|  192| 223| 413| 313| 430|  582|
|ОНЛАЙН Греческий ...| null| null| null| null| null|   60|  80|null|null|null| null|
|Архитектура Москв...|   94|   72|  122|   90|  100|   68|  50|  51|null|null|   14|
|ОНЛАЙН География....| 4251| 4702| 4862| 4007| 4724| 4013|2898|2719|3242|3651| 4244|
|Искусство коммуни...|  176|  153|  168|  139|  182|   61|  70|  65|  19|  45|   75|
|   История искусства|  766|  825| 1090|  952| 1210|  138| 269| 334| 362| 426|  703|
|Вышивка гладью и ...|    2|    9|    8|    4|    9| null|null|   1|null|null|    1|
|  Интернет-маркетинг|   64| null| null|   49|   59| null|null|null|null|null| null|
|ОНЛАЙН История, к...| 2934| 3195| 3690| 3679| 3682| 2889|2423|23

In [36]:
for i in range(1,13):
    pivot_1 = pivot_1.withColumnRenamed(f"{i}",f"p1_{i}")
    pivot_2 = pivot_2.withColumnRenamed(f"{i}",f"p2_{i}")

In [37]:
pivot_1.show()

+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|         направление| p1_1|p1_10|p1_11|p1_12| p1_2| p1_4| p1_5| p1_6| p1_7| p1_8| p1_9|
+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|                 ОФП|16060|17074|19792|17627|18703|11627| 9289| 8717| 6223| 7327|11807|
|   Иностранные языки| 5156| 4363| 5437| 4764| 5867| 1882| 1371|  972|  452|  544| 2052|
|          Киберспорт| null|   10|    8|    1|    1|    4|    1|    4| null|   19|   15|
|               Пение| 9153|10497|11795|10399|11421| 5894| 4816| 4523| 2975| 4050| 6957|
|ОНЛАЙН Художестве...|14889|14165|16573|16874|17783|14646|11722|11472| 9749|10360|12128|
|ОНЛАЙН Образовате...|15630|16208|17770|16881|18010|15826|12262|12153|10422|11517|14333|
|ОНЛАЙН Английский...|13415|14726|16315|16171|14907|19114|14461|11535| 8580| 8501|12556|
|    ОНЛАЙН Рисование|21511|22333|24782|25684|25197|21472|16997|16612|14897|17903|20666|
|ОНЛАЙН Шахматы и ...

In [38]:
pivot_2.show()

+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|         направление| p2_1|p2_10|p2_11|p2_12| p2_2| p2_4|p2_5|p2_6|p2_7|p2_8| p2_9|
+--------------------+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+
|Мастер-класс по у...|  603|  984|  956|  818|  818|  192| 223| 413| 313| 430|  582|
|ОНЛАЙН Греческий ...| null| null| null| null| null|   60|  80|null|null|null| null|
|Архитектура Москв...|   94|   72|  122|   90|  100|   68|  50|  51|null|null|   14|
|ОНЛАЙН География....| 4251| 4702| 4862| 4007| 4724| 4013|2898|2719|3242|3651| 4244|
|Искусство коммуни...|  176|  153|  168|  139|  182|   61|  70|  65|  19|  45|   75|
|   История искусства|  766|  825| 1090|  952| 1210|  138| 269| 334| 362| 426|  703|
|Вышивка гладью и ...|    2|    9|    8|    4|    9| null|null|   1|null|null|    1|
|  Интернет-маркетинг|   64| null| null|   49|   59| null|null|null|null|null| null|
|ОНЛАЙН История, к...| 2934| 3195| 3690| 3679| 3682| 2889|2423|23

In [70]:
pivot_summ = pivot_1.join(pivot_2, "направление", "full")
pivot_summ.show()

+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+
|         направление| p1_1|p1_10|p1_11|p1_12| p1_2|p1_4|p1_5|p1_6|p1_7|p1_8|p1_9|p2_1|p2_10|p2_11|p2_12|p2_2|p2_4|p2_5|p2_6|p2_7|p2_8|p2_9|
+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+
|"ОНЛАЙН Психологи...| null| null| null| null| null|null|null|null|null|null|null| 776|  872|  901|  883|1002| 440| 283| 285| 281| 565| 687|
|"Психологический ...| null| null| null| null| null|null|null|null|null|null|null| 127|   71|   89|  136| 139|  11|   4|null|null|   3|  23|
|    3D-моделирование| null| null| null| null| null|null|null|null|null|null|null|   4| null| null| null|  48|null|null|null|null|null|null|
| Автомобильная школа| null| null| null| null| null|null|null|null|null|null|null| 134|  133|  166|  144| 186|  92|  67|  53|null|null|  18|
|Авторская ку

In [71]:
pivot_summ = pivot_summ.na.fill(0)
pivot_summ.show()

+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+
|         направление| p1_1|p1_10|p1_11|p1_12| p1_2|p1_4|p1_5|p1_6|p1_7|p1_8|p1_9|p2_1|p2_10|p2_11|p2_12|p2_2|p2_4|p2_5|p2_6|p2_7|p2_8|p2_9|
+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+
|"ОНЛАЙН Психологи...|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0| 776|  872|  901|  883|1002| 440| 283| 285| 281| 565| 687|
|"Психологический ...|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0| 127|   71|   89|  136| 139|  11|   4|   0|   0|   3|  23|
|    3D-моделирование|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0|   4|    0|    0|    0|  48|   0|   0|   0|   0|   0|   0|
| Автомобильная школа|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0| 134|  133|  166|  144| 186|  92|  67|  53|   0|   0|  18|
|Авторская ку

In [72]:
for i in range(1,13):
    try:
        pivot_summ = pivot_summ.withColumn(f"{i}", col(f"p1_{i}")+col(f"p2_{i}"))
    except:
        continue
pivot_summ.show()

+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+----+----+----+----+----+----+-----+-----+-----+
|         направление| p1_1|p1_10|p1_11|p1_12| p1_2|p1_4|p1_5|p1_6|p1_7|p1_8|p1_9|p2_1|p2_10|p2_11|p2_12|p2_2|p2_4|p2_5|p2_6|p2_7|p2_8|p2_9|    1|    2|   4|   5|   6|   7|   8|   9|   10|   11|   12|
+--------------------+-----+-----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+-----+----+----+----+----+----+----+----+-----+-----+----+----+----+----+----+----+-----+-----+-----+
|"ОНЛАЙН Психологи...|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0| 776|  872|  901|  883|1002| 440| 283| 285| 281| 565| 687|  776| 1002| 440| 283| 285| 281| 565| 687|  872|  901|  883|
|"Психологический ...|    0|    0|    0|    0|    0|   0|   0|   0|   0|   0|   0| 127|   71|   89|  136| 139|  11|   4|   0|   0|   3|  23|  127|  139|  11|   4|   0|   0|   3|  23|   71|   89|  

In [73]:
for i in range(1,13):
    try:
        pivot_summ = pivot_summ.drop(f"p1_{i}")
    except:
        pass
    try:
        pivot_summ = pivot_summ.drop(f"p2_{i}")
    except:
        pass

pivot_summ.show()


+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+
|         направление|    1|    2|   4|   5|   6|   7|   8|   9|   10|   11|   12|
+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+
|"ОНЛАЙН Психологи...|  776| 1002| 440| 283| 285| 281| 565| 687|  872|  901|  883|
|"Психологический ...|  127|  139|  11|   4|   0|   0|   3|  23|   71|   89|  136|
|    3D-моделирование|    4|   48|   0|   0|   0|   0|   0|   0|    0|    0|    0|
| Автомобильная школа|  134|  186|  92|  67|  53|   0|   0|  18|  133|  166|  144|
|Авторская кукла. ...|    0|    0|   0|  20|  15|   0|   0|  10|    0|    0|    0|
|Авторская кукла. ...|    0|   40|   0|  80|  78|   0|   0|  92|  141|   20|    0|
|Авторские курсы/м...| 2159| 2297| 203| 180| 459| 965|1301|1581| 2234| 2527| 2177|
|Адаптивная и тони...| 1315| 1422| 694| 609| 659| 504| 476| 685| 1173| 1383| 1342|
|Акварельная живопись|  581|  740| 377| 312| 320| 114|  63| 286|  561|  675|  618|
|Анг

In [74]:
pivot_summ = pivot_summ.withColumn("total", col("1"))
for i in range(2,13):
    try:
        pivot_summ = pivot_summ.withColumn("total", col("total")+col(f"{i}"))
    except:
        continue
pivot_summ.show()

+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+
|         направление|    1|    2|   4|   5|   6|   7|   8|   9|   10|   11|   12| total|
+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+
|"ОНЛАЙН Психологи...|  776| 1002| 440| 283| 285| 281| 565| 687|  872|  901|  883|  6975|
|"Психологический ...|  127|  139|  11|   4|   0|   0|   3|  23|   71|   89|  136|   603|
|    3D-моделирование|    4|   48|   0|   0|   0|   0|   0|   0|    0|    0|    0|    52|
| Автомобильная школа|  134|  186|  92|  67|  53|   0|   0|  18|  133|  166|  144|   993|
|Авторская кукла. ...|    0|    0|   0|  20|  15|   0|   0|  10|    0|    0|    0|    45|
|Авторская кукла. ...|    0|   40|   0|  80|  78|   0|   0|  92|  141|   20|    0|   451|
|Авторские курсы/м...| 2159| 2297| 203| 180| 459| 965|1301|1581| 2234| 2527| 2177| 16083|
|Адаптивная и тони...| 1315| 1422| 694| 609| 659| 504| 476| 685| 1173| 1383| 1342| 10262|
|Акварельн

In [75]:
pivot_summ = pivot_summ.withColumn("mean", col("total")/12)
pivot_summ.show()

+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+------------------+
|         направление|    1|    2|   4|   5|   6|   7|   8|   9|   10|   11|   12| total|              mean|
+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+------------------+
|"ОНЛАЙН Психологи...|  776| 1002| 440| 283| 285| 281| 565| 687|  872|  901|  883|  6975|            581.25|
|"Психологический ...|  127|  139|  11|   4|   0|   0|   3|  23|   71|   89|  136|   603|             50.25|
|    3D-моделирование|    4|   48|   0|   0|   0|   0|   0|   0|    0|    0|    0|    52| 4.333333333333333|
| Автомобильная школа|  134|  186|  92|  67|  53|   0|   0|  18|  133|  166|  144|   993|             82.75|
|Авторская кукла. ...|    0|    0|   0|  20|  15|   0|   0|  10|    0|    0|    0|    45|              3.75|
|Авторская кукла. ...|    0|   40|   0|  80|  78|   0|   0|  92|  141|   20|    0|   451|37.583333333333336|
|Авторские курсы/м.

In [77]:
for i in range(1,13):
    try:
        pivot_summ = pivot_summ.withColumn(f"mean_dev_{i}", col(f"{i}")/col("mean"))
    except:
        pass
pivot_summ.show()

+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+
|         направление|    1|    2|   4|   5|   6|   7|   8|   9|   10|   11|   12| total|              mean|        mean_dev_1|        mean_dev_2|         mean_dev_4|         mean_dev_5|         mean_dev_6|         mean_dev_7|         mean_dev_8|         mean_dev_9|       mean_dev_10|       mean_dev_11|        mean_dev_12|
+--------------------+-----+-----+----+----+----+----+----+----+-----+-----+-----+------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+
|"ОНЛАЙН Психологи...|  7

In [78]:
df = pivot_summ.toPandas()

In [79]:
df.head()

,направление,1,2,4,5,6,7,8,9,10,11,12,total,mean,mean_dev_1,mean_dev_2,mean_dev_4,mean_dev_5,mean_dev_6,mean_dev_7,mean_dev_8,mean_dev_9,mean_dev_10,mean_dev_11,mean_dev_12
0,"""ОНЛАЙН Психологический лекторий """"Всё только ...",776,1002,440,283,285,281,565,687,872,901,883,6975,581.250000,1.335054,1.723871,0.756989,0.486882,0.490323,0.483441,0.972043,1.181935,1.500215,1.550108,1.519140
1,"""Психологический лекторий """"Всё только начинае...",127,139,11,4,0,0,3,23,71,89,136,603,50.250000,2.527363,2.766169,0.218905,0.079602,0.000000,0.000000,0.059701,0.457711,1.412935,1.771144,2.706468
2,3D-моделирование,4,48,0,0,0,0,0,0,0,0,0,52,4.333333,0.923077,11.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Автомобильная школа,134,186,92,67,53,0,0,18,133,166,144,993,82.750000,1.619335,2.247734,1.111782,0.809668,0.640483,0.000000,0.000000,0.217523,1.607251,2.006042,1.740181
4,Авторская кукла. Базовый курс,0,0,0,20,15,0,0,10,0,0,0,45,3.750000,0.000000,0.000000,0.000000,5.333333,4.000000,0.000000,0.000000,2.666667,0.000000,0.000000,0.000000


In [80]:
df.to_csv("monts_destribution.csv", encoding="utf-8")